In [1]:
from plotly.offline import download_plotlyjs, init_notebook_mode, plot, iplot
init_notebook_mode(connected=True)

In [2]:
import pandas

# Note the 'set_index' call !! 
# This is necessary for filtering out sentences with incorrect NER annotations:
train = pandas.read_csv(r'C:\Users\JYellin\re_1\tacred\results\general\train-ud_paths_v0.0.5.csv').set_index(['id'])
test = pandas.read_csv(r'C:\Users\JYellin\re_1\tacred\results\general\test-ud_paths_v0.0.5.csv').set_index(['id'])

In [3]:
train_ner = pandas.read_csv(r'C:\Users\JYellin\re_1\tacred\results\general\train-ud_paths_with_ner_v0.0.6.csv').set_index(['id'])
test_ner = pandas.read_csv(r'C:\Users\JYellin\re_1\tacred\results\general\test-ud_paths_with_ner_v0.0.6.csv').set_index(['id'])

train_ids_with_proper_ner = train_ner[(train_ner['type1']==train_ner['type1_corenlp']) & (train_ner['type2']==train_ner['type2_corenlp'])].drop(['docid', 'tokens', 'relation', 'path', 'lemmas_on_path', 'type1', 'type2', 'ent1_head', 'ent2_head', 'type1_corenlp', 'type2_corenlp'], axis=1)
test_ids_with_proper_ner = test_ner[(test_ner['type1']==test_ner['type1_corenlp']) & (test_ner['type2']==test_ner['type2_corenlp'])].drop(['docid', 'tokens', 'relation', 'path', 'lemmas_on_path', 'type1', 'type2', 'ent1_head', 'ent2_head', 'type1_corenlp', 'type2_corenlp'], axis=1)

In [4]:
train = train_ids_with_proper_ner.join(train).dropna(axis=0).reset_index()
test = test_ids_with_proper_ner.join(test).dropna(axis=0).reset_index()

In [5]:
print('Train shape ', train.shape)
print('Test shape ', test.shape)

Train shape  (56648, 10)
Test shape  (12839, 10)


In [6]:
def transform_row(r):
    return '{entity1} {org_path} {entity2}'.format(entity1=r.type1[0:3], org_path=r.path, entity2=r.type2[0:3])

train['path'] = train.apply(transform_row, axis=1)
test['path'] = test.apply(transform_row, axis=1)

In [7]:
relation_to_paths = train.groupby(['relation'])['path'].unique().to_dict()

In [8]:
def label_sentence_recall(test_row, relation):
    path = test_row['path']
    true_relation = test_row['relation']
    
    if path in relation_to_paths[relation]:
        if true_relation == relation:
            return 'TP'
        else:
            return 'FP'
    else:
        return 'N'

In [9]:
for relation in filter(lambda relation: relation != 'no_relation', relation_to_paths.keys()):
    test[relation] = test.apply(lambda row : label_sentence_recall(row, relation), axis=1)    

In [14]:
stats = []
overall_tp = 0
overall_fp = 0

for relation in filter(lambda relation: relation != 'no_relation', relation_to_paths.keys()):

    counts = test[relation].value_counts().to_dict()
        
    tp = counts.get('TP', 0)
    fp = counts.get('FP', 0)
    
    if (tp+fp) > 0:
        precision = tp / (tp+fp)
    else:
        precision = 0

    stats.append( (relation, round(precision,4)) )
    
    overall_tp += tp
    overall_fp += fp
    
overall_precision = overall_tp / (overall_tp+overall_fp)

    

stats.append( ('overall', round(overall_precision,4)) )
stats = sorted(stats, key=lambda x: x[1], reverse=True)

In [15]:
import plotly.graph_objs as go

overall_index = next(i for (i, (relation, score)) in enumerate(stats) if relation == 'overall')

colors = ['rgb(58,200,225)'] * len(stats)
colors[overall_index] = 'rgb(181,59,89)'


trace1 = go.Bar(
    x=[stat[0] for stat in stats],
    y=[stat[1] for stat in stats],
    name='Precision',
    width=0.5,
    marker=dict(
        color=colors,
        line=dict(
            color='rgb(8,48,107)',
            width=0.5),
        ),
    opacity=0.6
)

data = [trace1]
layout = go.Layout(
    title='UD Path with Vetted Entity Type - Precision',
    xaxis_title="Relation",
    yaxis_title="Precision ",        
    barmode='overlay',
    width=1200,
    xaxis = go.layout.XAxis(
        tickangle = 45,
        automargin = True
        
    )
    
        
)

fig = go.Figure(data=data, layout=layout)
iplot(fig)

In [ ]:
stats